# Phase 3: Preprocessing & Feature Engineering

This notebook explores the preprocessing pipeline that transforms raw Atari frames into a format suitable for deep reinforcement learning.

## Objectives
1. Understand each preprocessing step
2. Compare raw vs preprocessed observations
3. Visualize the transformation pipeline
4. Test preprocessed environment
5. Validate preprocessing benefits

In [ ]:
import sys
sys.path.append('../src')

import gymnasium as gym
import ale_py
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import json
from pathlib import Path

from preprocessing import make_atari_env, WarpFrame, FrameStack, ClipRewardEnv

# Register ALE environments
gym.register_envs(ale_py)

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("✓ Imports successful")
print("✓ Preprocessing module loaded")

## 1. Raw vs Preprocessed Observations

In [ ]:
# Create raw environment
env_raw = gym.make('ALE/SpaceInvaders-v5', render_mode='rgb_array')
obs_raw, _ = env_raw.reset()

# Create preprocessed environment
env_preprocessed = make_atari_env('ALE/SpaceInvaders-v5', frame_stack=4, clip_rewards=True)
obs_preprocessed, _ = env_preprocessed.reset()

print("OBSERVATION COMPARISON")
print("=" * 60)
print(f"\nRaw Observation:")
print(f"  Shape: {obs_raw.shape}")
print(f"  Dtype: {obs_raw.dtype}")
print(f"  Memory: {obs_raw.nbytes / 1024:.2f} KB")
print(f"  Range: [{obs_raw.min()}, {obs_raw.max()}]")

print(f"\nPreprocessed Observation:")
print(f"  Shape: {obs_preprocessed.shape}")
print(f"  Dtype: {obs_preprocessed.dtype}")
print(f"  Memory: {obs_preprocessed.nbytes / 1024:.2f} KB")
print(f"  Range: [{obs_preprocessed.min():.2f}, {obs_preprocessed.max():.2f}]")

# Calculate reductions
memory_reduction = (1 - obs_preprocessed.nbytes / obs_raw.nbytes) * 100
raw_dims = np.prod(obs_raw.shape)
proc_dims = np.prod(obs_preprocessed.shape)
dim_reduction = (1 - proc_dims / raw_dims) * 100

print(f"\n💾 REDUCTION:")
print(f"  Memory: {memory_reduction:.1f}%")
print(f"  Dimensions: {raw_dims:,} → {proc_dims:,} ({dim_reduction:.1f}% reduction)")

## 2. Visualize Preprocessing Pipeline

In [ ]:
# Get a more interesting frame
env = gym.make('ALE/SpaceInvaders-v5', render_mode='rgb_array')
obs, _ = env.reset()
for _ in range(30):
    obs, _, _, _, _ = env.step(env.action_space.sample())

# Apply each preprocessing step
obs_gray = cv2.cvtColor(obs, cv2.COLOR_RGB2GRAY)
obs_resized = cv2.resize(obs_gray, (84, 84), interpolation=cv2.INTER_AREA)
obs_normalized = obs_resized.astype(np.float32) / 255.0

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

axes[0, 0].imshow(obs)
axes[0, 0].set_title(f'Step 1: Raw RGB\nShape: {obs.shape}', fontsize=12, fontweight='bold')
axes[0, 0].axis('off')

axes[0, 1].imshow(obs_gray, cmap='gray')
axes[0, 1].set_title(f'Step 2: Grayscale\nShape: {obs_gray.shape}', fontsize=12, fontweight='bold')
axes[0, 1].axis('off')

axes[1, 0].imshow(obs_resized, cmap='gray')
axes[1, 0].set_title(f'Step 3: Resized to 84×84\nShape: {obs_resized.shape}', fontsize=12, fontweight='bold')
axes[1, 0].axis('off')

axes[1, 1].imshow(obs_normalized, cmap='gray', vmin=0, vmax=1)
axes[1, 1].set_title(f'Step 4: Normalized [0,1]\nShape: {obs_normalized.shape}', fontsize=12, fontweight='bold')
axes[1, 1].axis('off')

plt.suptitle('Preprocessing Pipeline', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

env.close()

## 3. Frame Stacking Visualization

In [ ]:
# Create environment with frame stacking
env_stacked = make_atari_env('ALE/SpaceInvaders-v5', frame_stack=4)
obs_stacked, _ = env_stacked.reset()

# Take some actions to populate frame stack
for _ in range(30):
    obs_stacked, _, _, _, _ = env_stacked.step(env_stacked.action_space.sample())

print(f"Frame Stack Shape: {obs_stacked.shape}")
print(f"This represents 4 consecutive frames stacked along the channel dimension")

# Visualize all 4 frames
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for i in range(4):
    axes[i].imshow(obs_stacked[:, :, i], cmap='gray', vmin=0, vmax=1)
    axes[i].set_title(f'Frame {i+1} (t-{3-i})', fontsize=12, fontweight='bold')
    axes[i].axis('off')

plt.suptitle('Frame Stack: 4 Consecutive Frames', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n💡 Frame stacking captures motion:")
print("   - Frame 1 (t-3): oldest frame")
print("   - Frame 4 (t-0): current frame")
print("   - Agent can infer velocity and direction from multiple frames")

env_stacked.close()

## 4. Reward Clipping Analysis

In [ ]:
# Test reward clipping
print("REWARD CLIPPING DEMONSTRATION")
print("=" * 60)

# Sample rewards and their clipped versions
sample_rewards = [0, 5, 10, 15, 20, 40, 100, -5, -10]
clipped_rewards = [np.sign(r) for r in sample_rewards]

print("\nOriginal → Clipped")
print("-" * 30)
for orig, clipped in zip(sample_rewards, clipped_rewards):
    print(f"  {orig:4d} → {clipped:2.0f}")

print("\n💡 Benefits of Reward Clipping:")
print("   ✓ Reduces variance in gradients")
print("   ✓ Makes learning more stable")
print("   ✓ Treats all positive rewards equally")
print("   ✓ Standard practice for DQN")

## 5. Load and Analyze Preprocessing Results

In [ ]:
# Load preprocessing info
with open('../data/phase3/preprocessing_info.json', 'r') as f:
    preprocessing_info = json.load(f)

print("PREPROCESSING PIPELINE STEPS")
print("=" * 60)
for step, description in preprocessing_info['preprocessing_steps'].items():
    step_num = step.split('_')[0]
    step_name = ' '.join(step.split('_')[1:]).title()
    print(f"\n{step_num}. {step_name}")
    print(f"   {description}")

print("\n\nKEY BENEFITS")
print("=" * 60)
for benefit, description in preprocessing_info['benefits'].items():
    print(f"✓ {benefit.replace('_', ' ').title()}: {description}")

## 6. Performance Comparison

In [ ]:
# Load performance stats
perf_stats = preprocessing_info['performance_stats']

print("PREPROCESSED ENVIRONMENT PERFORMANCE")
print("=" * 60)
print(f"Episodes Tested: {perf_stats['num_episodes']}")
print(f"Mean Reward (clipped): {perf_stats['mean_reward']:.2f} ± {perf_stats['std_reward']:.2f}")
print(f"Mean Episode Length: {perf_stats['mean_length']:.1f} ± {perf_stats['std_length']:.1f}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

episodes = range(1, len(perf_stats['episode_rewards']) + 1)

# Rewards
axes[0].plot(episodes, perf_stats['episode_rewards'], 'o-', alpha=0.6, color='steelblue')
axes[0].axhline(y=perf_stats['mean_reward'], color='red', linestyle='--', 
                label=f"Mean: {perf_stats['mean_reward']:.2f}", linewidth=2)
axes[0].set_xlabel('Episode', fontsize=12)
axes[0].set_ylabel('Total Reward (Clipped)', fontsize=12)
axes[0].set_title('Preprocessed Environment - Episode Rewards', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Lengths
axes[1].bar(episodes, perf_stats['episode_lengths'], alpha=0.7, color='coral', edgecolor='black')
axes[1].axhline(y=perf_stats['mean_length'], color='red', linestyle='--', 
                label=f"Mean: {perf_stats['mean_length']:.1f}", linewidth=2)
axes[1].set_xlabel('Episode', fontsize=12)
axes[1].set_ylabel('Episode Length', fontsize=12)
axes[1].set_title('Preprocessed Environment - Episode Lengths', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Key Insights

### Preprocessing Achievements:

1. **Dimensionality Reduction**: 72% reduction in pixel dimensions (100,800 → 28,224)
2. **Memory Efficiency**: Reduced memory footprint per observation
3. **Temporal Information**: Frame stacking captures motion and velocity
4. **Training Stability**: Reward clipping reduces gradient variance
5. **Standardization**: Follows DQN Nature paper methodology

### Why This Matters:

- ✅ **Faster Training**: Smaller observations = faster neural network forward/backward passes
- ✅ **Better Learning**: Frame stacking provides temporal context
- ✅ **Stability**: Reward clipping prevents gradient explosion
- ✅ **Reproducibility**: Standard preprocessing allows fair comparisons

### Next Steps:

With preprocessing pipeline complete, we're ready to implement our first DQN agent in Phase 4!

## Summary

✅ Phase 3 Complete!

We have:
- Implemented comprehensive preprocessing pipeline
- Reduced observation dimensions by 72%
- Added temporal information via frame stacking
- Implemented reward clipping for stability
- Created reusable preprocessing utilities
- Validated preprocessing with random agent

**Ready for Phase 4: DQN Agent - Part 1 (Basic Implementation)**